# Decoder only Summarizer

Example of using a decoder-only language model from Hugging Face's Transformers library for text summarization in PyTorch. We'll be using the BartForConditionalGeneration model, which is a pre-trained sequence-to-sequence model based on the BART (Bidirectional and Auto-Regressive Transformers) architecture.

In [2]:
!pip install transformers --quiet

We import the required modules from the Transformers library: BartForConditionalGeneration (the pre-trained model) and BartTokenizer (for tokenizing the input text)

In [3]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer


We load the pre-trained bart-large-cnn model and tokenizer using the from_pretrained method

In [4]:
# Load the pre-trained model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

We move the model to the GPU if available, using PyTorch's device functionality

In [5]:
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Example article

In [11]:
# Example article to summarize
article = """
The Astonishing Hypothesis is a 1994 book by scientist Francis Crick about consciousness and neuroscience. In it, Crick promotes the idea that consciousness is produced by physical and chemical processes in the brain, and that neuroscience will eventually have a theory which can explain consciousness. The book surveys the history of research into consciousness and outlines several hypotheses about the neural correlates of various components and properties of consciousness.
"""

We tokenize the article using the tokenizer, specifying that we want the output as PyTorch tensors (return_tensors="pt"). 

We also set max_length=1024 to truncate the input if it exceeds that length, and truncation=True to allow truncation

In [7]:
# Tokenize the article
inputs = tokenizer(article, return_tensors="pt", max_length=1024, truncation=True)
input_ids = inputs.input_ids.to(device)

We call the generate method of the model with the tokenized input. 

We specify num_beams=4 to use beam search with 4 beams, max_length=100 to limit the summary length to 100 tokens, and early_stopping=True to stop generating once the model produces the end-of-sequence token

In [8]:
# Generate the summary
summary_ids = model.generate(input_ids, num_beams=4, max_length=100, early_stopping=True)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


The generate method returns a tensor of token IDs representing the generated summary. We use the tokenizer.decode method to convert the token IDs back into text, passing skip_special_tokens=True to exclude any special tokens (e.g., start/end-of-sequence tokens) from the output

In [9]:
# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [12]:
print(f"Summary: {summary}")

Summary: The Astonishing Hypothesis is a 1994 book by scientist Francis Crick about consciousness and neuroscience. In it, Crick promotes the idea that consciousness is produced by physical and chemical processes in the brain, and that neuroscience will eventually have a theory which can explain consciousness.
